In [2]:
abstract type Node end
abstract type Operator end
abstract type LeafNode <: Node end

In [3]:
mutable struct Variable{T} <: LeafNode
    value::T
    grad::T
    Variable(val::T) where T          = new{T}(val, zero(val))
    Variable(val::T, grad::T) where T = new{T}(val, grad)
end

In [4]:
struct Method{OT} <: Operator
    f::OT
end

struct Broadcasted{OT} <: Operator
    f::OT
end

In [5]:
struct ComputableNode{OT <: Operator, AT <: Tuple, KT <: NamedTuple} <: Node
    op::OT
    args::AT
    kwargs::KT
end
ComputableNode(op::Function, args, kwargs) = ComputableNode(Method(op), args, kwargs)
ComputableNode(op, args)                   = ComputableNode(op, args, NamedTuple())

ComputableNode

In [6]:
mutable struct CachedNode{NT <: Node, OUT} <: Node
    node::NT
    out::OUT
end

function CachedNode(op, args...; kwargs...)
    node = ComputableNode(op, args, kwargs.data)
    out  = forward(node)
    CachedNode(node, out)
end

CachedNode

In [7]:
arg(x::ComputableNode, i::Int) = x.args[i]
args(x::ComputableNode) = x.args
kwargs(x::ComputableNode) = x.kwargs
operator(x::ComputableNode) = x.f

arg(x::CachedNode, i::Int) = x.node.args[i]
args(x::CachedNode) = x.node.args
kwargs(x::CachedNode) = x.node.kwargs
operator(x::CachedNode) = x.node.f

operator (generic function with 2 methods)

In [8]:
import Base: show
show(io::IO, x::Method)         = print(io, "fn ",  x.f);
show(io::IO, x::Operator)       = print(io, "op ",  x.f);
show(io::IO, x::Variable)       = print(io, "var ", x.value);
show(io::IO, x::CachedNode)     = print(io, "{cached (", x.node, ") => ", x.out, "}");
show(io::IO, x::ComputableNode) = print(io, "[", x.op, "](", x.args, ")");

In [9]:
forward(x) = x
forward(leaf::LeafNode) = value(leaf)
forward(node::ComputableNode) = forward(node.op, map(forward, node.args)...; map(forward, node.kwargs)...)
forward(cached::CachedNode) = (cached.out = forward(cached.node))
forward(op::Broadcasted, args...) = Broadcast.broadcasted(op.f, args...)
forward(op::Operator, args...; kwargs...) = op.f(args...; kwargs...)
forward(x::NT) where {NT <: Node} = error("forward method is not implemented for node type: $NT")

forward (generic function with 7 methods)

In [10]:
value(x) = x
value(x::Variable) = x.value
value(x::CachedNode) = value(x.out)
value(x::T) where {T <: Node} = error("Expected value in this node $x of type $T
 check if you defined a non-cached node
 or overload value function for your node.")

value (generic function with 4 methods)

In [11]:
function backward(x::Variable, grad)
    if isdefined(x, :grad)
        x.grad+= grad
    else
        x.grad = grad
    end
    nothing
end

function backward(node::CachedNode, f, grad)
    grad_inputs = gradient(node, grad)
    for (each, each_grad) in zip(args(node), grad_inputs)
        backward(each, each_grad)
    end
    nothing
end

function backward(cached::CachedNode, op::Broadcasted, grad)
    grad_inputs = gradient(cached, grad)
    for (each, each_grad) in zip(args(cached), grad_inputs)
        backward(each, each_grad)
    end
    nothing
end

backward(cached::CachedNode) = backward(cached, 1.0)
backward(cached::CachedNode, grad) = backward(cached, cached.node.op, grad)
backward(cached::CachedNode, op::Method, grad) = backward(cached, op.f, grad)
backward(cached::CachedNode, ::typeof(Broadcast.materialize), grad) = backward(arg(cached, 1), grad)

backward (generic function with 7 methods)

In [12]:
gradient(x::CachedNode, grad) = gradient(x.node.op, grad, x.out, map(value, x.node.args)...; map(value, x.node.kwargs)...)
gradient(x::Operator,   grad, out, args...; kwargs...) = gradient(x.f, grad, out, args...; kwargs...)
gradient(op, grad, out, args...; kwargs...) = error("gradient of operator $op is not defined\n
 Possible Fix:\n
 define one of the following:\n
 1. gradient(::typeof($op), grad, out, args...; kwargs...)\n
 2. gradient(op::Method{typeof($op)}, grad, out, args...; kwargs...)\n
 3. gradient(op::Broadcasted{typeof($op)}, grad, out, args...; kwargs...)\n")

gradient (generic function with 3 methods)

In [13]:
import Base: +, -, *, /
+(x::Node) = CachedNode(+, x)
-(x::Node) = CachedNode(-, x)
gradient(::typeof(+), grad, output, x) = (grad * 1, )
gradient(::typeof(-), grad, output, x) = (grad *-1, )
+(x::Node, y::Node) = CachedNode(+, x, y)
-(x::Node, y::Node) = CachedNode(-, x, y)
*(x::Node, y::Node) = CachedNode(*, x, y)
/(x::Node, y::Node) = CachedNode(/, x, y)
gradient(::typeof(+), grad, output, x, y) = (grad * one(x),   grad * one(y))
gradient(::typeof(-), grad, output, x, y) = (grad * one(x),   grad *-one(y))
gradient(::typeof(*), grad, output, x, y) = (grad * y,        grad * x)
gradient(::typeof(/), grad, output, x, y) = (grad * one(x)/y, grad *-x/y/y)

gradient (generic function with 9 methods)

In [14]:
import Base: abs, sin, cos, tan, exp, sqrt, isless
abs(x::Node)  = CachedNode(abs, x)
sin(x::Node)  = CachedNode(sin, x)
cos(x::Node)  = CachedNode(cos, x)
tan(x::Node)  = CachedNode(tan, x)
exp(x::Node)  = CachedNode(exp, x)
sqrt(x::Node) = CachedNode(sqrt, x)
isless(x::Node, y::Node) = isless(value(x), value(y))
gradient(::typeof(abs), grad, output, x)  = (grad * sign(x), )
gradient(::typeof(sin), grad, output, x)  = (grad * cos(x), )
gradient(::typeof(cos), grad, output, x)  = (grad *-sin(x), )
gradient(::typeof(tan), grad, output, x)  = (grad *(tan(x)^2 + 1), )
gradient(::typeof(exp), grad, output, x)  = (grad * exp(x), )
gradient(::typeof(sqrt), grad, output, x) = (grad * 0.5/sqrt(x), )

gradient (generic function with 15 methods)

In [15]:
import Base: convert, promote_rule
convert(::Type{Variable{T}}, x::Number) where T   = Variable(convert(T, x))
convert(::Type{Variable{T}}, x::Variable) where T = Variable(convert(T, x.value), convert(T, x.grad))
promote_rule(::Type{Variable{T}}, ::Type{R}) where {T,R} = Variable{promote_type(R,T)}

promote_rule (generic function with 123 methods)

In [16]:
struct ComputGraphStyle <: Broadcast.BroadcastStyle end
Base.BroadcastStyle(::Type{<:Node}) = ComputGraphStyle()
Broadcast.BroadcastStyle(s::ComputGraphStyle, x::Broadcast.BroadcastStyle) = s
Broadcast.broadcasted(::ComputGraphStyle, f, args...) = CachedNode(Broadcasted(f), args...)
Broadcast.broadcastable(x::Node) = x
Broadcast.materialize(x::Node) = CachedNode(Broadcast.materialize, x)
Base.similar(x::Node)                                      = Variable(similar(value(x)))
Base.similar(x::Node, dims::Dims)                          = Variable(similar(value(x), dims))
Base.similar(x::Node, eltype::Type{S}, dims::Dims) where S = Variable(similar(value(x), eltype, dims))

In [17]:
gradient(::Broadcasted{typeof(+)}, grad, output, x)    = @. (grad * 1, )
gradient(::Broadcasted{typeof(-)}, grad, output, x)    = @. (grad *-1, )
gradient(::Broadcasted{typeof(+)}, grad, output, x, y) = @. (grad * one(x),   grad * one(y))
gradient(::Broadcasted{typeof(-)}, grad, output, x, y) = @. (grad * one(x),   grad *-one(y))
gradient(::Broadcasted{typeof(*)}, grad, output, x, y) = @. (grad * y,        grad * x)
gradient(::Broadcasted{typeof(/)}, grad, output, x, y) = @. (grad * one(x)/y, grad *-x/y/y)
gradient(::Broadcasted{typeof(abs)}, grad, output, x)  = @. (grad * sign(x), )
gradient(::Broadcasted{typeof(sin)}, grad, output, x)  = @. (grad *  cos(x),  )
gradient(::Broadcasted{typeof(cos)}, grad, output, x)  = @. (grad * -sin(x), )
gradient(::Broadcasted{typeof(tan)}, grad, output, x)  = @. (grad * (tan(x)^2 + 1), )
gradient(::Broadcasted{typeof(exp)}, grad, output, x)  = @. (grad *  exp(x), )
gradient(::Broadcasted{typeof(sqrt)}, grad, output, x) = @. (grad *.5/sqrt(x), )

gradient (generic function with 27 methods)

In [18]:
x = Variable(1.0)
z = sin(x*x)
backward(z)
@show x.grad;

x.grad = 1.0806046117362795


In [19]:
x = Variable(5.0)
z = sin(x*x)
backward(z)
@show x.grad;

x.grad = 9.912028118634735


In [ ]:
x = Variable(collect(-π:0.01:+π))
z = sin.(x.*x)
backward(z)

In [ ]:
using Plots
plot(value(x), value(z))
plot!(value(x), x.grad)

In [ ]:
import Base: iterate, sum
iterate(x::Node)     = iterate_forward(iterate(value(x)),     x)
iterate(x::Node, st) = iterate_forward(iterate(value(x), st), x, st)

function iterate_forward(out, x::Node, st)
    node = ComputableNode(iterate, (x, st))
    v, st = out
    CachedNode(node, v), st
end

function iterate_forward(out, x::Node)
    node = ComputableNode(iterate, (x, ))
    v, st = out
    CachedNode(node, v), st
end

function gradient(::typeof(iterate), grad, output, x::AbstractArray)
    out_grad = zero(x)
    out_grad[1] = grad
    (out_grad, )
end

function gradient(::typeof(iterate), grad, output, x::AbstractArray, st)
    out_grad = zero(x)
    out_grad[st] = grad
    (out_grad, )
end

sum(x::Variable{<:AbstractArray}; dims=:) = CachedNode(sum, x; dims=dims)
gradient(::typeof(sum), grad, output, x::AbstractArray; dims) = grad_sum(grad, x, dims)
grad_sum(grad, x, dims::Colon) = (fill!(similar(x), grad), )

In [31]:
x = Variable([1., 2.])
y = Variable([1.,-1.])
z = Variable([100.,100.])
u = Variable([1.,1.])
w = (u - x*x) + z*(y - x*x)*(y - x*x)
backward(w)
x.grad

ErrorException: gradient of operator * is not defined

 Possible Fix:

 define one of the following:

 1. gradient(::typeof(*), grad, out, args...; kwargs...)

 2. gradient(op::Method{typeof(*)}, grad, out, args...; kwargs...)

 3. gradient(op::Broadcasted{typeof(*)}, grad, out, args...; kwargs...)
